In [35]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo

In [36]:
#set the url as a variable
url = 'https://www.the-numbers.com/weekly-box-office-chart'

In [86]:
#make a call to the internet page
response = requests.get(url)
#parse the text version of the response using an html parser 
soup = bs(response.text, 'html.parser')
#pretty print the parsed text
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
  </script>
  <meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
  <!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="telephone=no" name="format-detection"/>
  <!-- for apple mobile -->
  <meta content="521546213" property="fb:admins">
   <meta content="initial-scale=1" name="viewport"/>
   <meta content="Weekly (

In [87]:
#finds the table in the HTML page
all_table_data = soup.find_all('table')
#finds and isolates only the <tr> tags and subtags within the above table
rows = soup.find_all('tr')
#start an empty master list of rows
data = []
#loop through each row
for row in rows:
    #each cell is a column within that row, so you use <td> to find each cell
    cols = row.find_all('td')
    #add the cells to a preliminary list using list comprehensions (this is is for a single row)
    cols = [element.text.strip() for element in cols]
    #add that preliminary list to the master list of rows
    data.append([element for element in cols])

In [88]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,← Previous Chart,Chart Index,,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None
2,1,(1),Aquaman,Warner Bros.,"$41,575,901",-55%,"4,184","$9,937","$270,596,781",3
3,2,new,Escape Room,Sony Pictures,"$23,532,931",,"2,717","$8,661","$23,532,931",1
4,3,(2),Mary Poppins Returns,Walt Disney,"$20,485,680",-61%,"4,090","$5,009","$143,441,985",4


In [89]:
df.drop([0,1], inplace=True)

In [90]:
df.columns = ["Rank", "Last Rank", "Movie", "Distributor", "Gross", "Change", "Theatres", "Per Theatre", "Total Gross", "Week"]

In [91]:
df.head()

,Rank,Last Rank,Movie,Distributor,Gross,Change,Theatres,Per Theatre,Total Gross,Week
2,1,(1),Aquaman,Warner Bros.,"$41,575,901",-55%,"4,184","$9,937","$270,596,781",3
3,2,new,Escape Room,Sony Pictures,"$23,532,931",,"2,717","$8,661","$23,532,931",1
4,3,(2),Mary Poppins Returns,Walt Disney,"$20,485,680",-61%,"4,090","$5,009","$143,441,985",4
5,4,(4),Spider-Man: Into The Spider…,Sony Pictures,"$17,924,099",-50%,"3,419","$5,242","$138,775,445",4
6,5,(3),Bumblebee,Paramount Pictures,"$17,341,837",-54%,"3,597","$4,821","$101,694,977",3


In [92]:
# Make a copy of dataframe
new_df = df.copy()

# remove all dollar signs
new_df['Gross'] = [x.strip('$') for x in df['Gross']]
new_df['Per Theatre'] = [x.strip('$') for x in df['Per Theatre']]
new_df['Total Gross'] = [x.strip('$') for x in df['Total Gross']]

new_df.head()

,Rank,Last Rank,Movie,Distributor,Gross,Change,Theatres,Per Theatre,Total Gross,Week
2,1,(1),Aquaman,Warner Bros.,"41,575,901",-55%,"4,184","9,937","270,596,781",3
3,2,new,Escape Room,Sony Pictures,"23,532,931",,"2,717","8,661","23,532,931",1
4,3,(2),Mary Poppins Returns,Walt Disney,"20,485,680",-61%,"4,090","5,009","143,441,985",4
5,4,(4),Spider-Man: Into The Spider…,Sony Pictures,"17,924,099",-50%,"3,419","5,242","138,775,445",4
6,5,(3),Bumblebee,Paramount Pictures,"17,341,837",-54%,"3,597","4,821","101,694,977",3


In [93]:
new_df['Gross'] = new_df['Gross'].str.replace(',', '')
new_df['Theatres'] = new_df['Theatres'].str.replace(',', '')
new_df['Total Gross'] = new_df['Total Gross'].str.replace(',', '')
new_df.head()

,Rank,Last Rank,Movie,Distributor,Gross,Change,Theatres,Per Theatre,Total Gross,Week
2,1,(1),Aquaman,Warner Bros.,41575901,-55%,4184,"9,937",270596781,3
3,2,new,Escape Room,Sony Pictures,23532931,,2717,"8,661",23532931,1
4,3,(2),Mary Poppins Returns,Walt Disney,20485680,-61%,4090,"5,009",143441985,4
5,4,(4),Spider-Man: Into The Spider…,Sony Pictures,17924099,-50%,3419,"5,242",138775445,4
6,5,(3),Bumblebee,Paramount Pictures,17341837,-54%,3597,"4,821",101694977,3


In [97]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [98]:
db = client.moviesdb

In [99]:
movies = db.movies

In [100]:
data = new_df.to_dict("records")

In [101]:
data

[{'Rank': '1',
  'Last Rank': '(1)',
  'Movie': 'Aquaman',
  'Distributor': 'Warner Bros.',
  'Gross': '41575901',
  'Change': '-55%',
  'Theatres': '4184',
  'Per Theatre': '9,937',
  'Total Gross': '270596781',
  'Week': '3'},
 {'Rank': '2',
  'Last Rank': 'new',
  'Movie': 'Escape Room',
  'Distributor': 'Sony Pictures',
  'Gross': '23532931',
  'Change': '',
  'Theatres': '2717',
  'Per Theatre': '8,661',
  'Total Gross': '23532931',
  'Week': '1'},
 {'Rank': '3',
  'Last Rank': '(2)',
  'Movie': 'Mary Poppins Returns',
  'Distributor': 'Walt Disney',
  'Gross': '20485680',
  'Change': '-61%',
  'Theatres': '4090',
  'Per Theatre': '5,009',
  'Total Gross': '143441985',
  'Week': '4'},
 {'Rank': '4',
  'Last Rank': '(4)',
  'Movie': 'Spider-Man: Into The Spider…',
  'Distributor': 'Sony Pictures',
  'Gross': '17924099',
  'Change': '-50%',
  'Theatres': '3419',
  'Per Theatre': '5,242',
  'Total Gross': '138775445',
  'Week': '4'},
 {'Rank': '5',
  'Last Rank': '(3)',
  'Movie': 'B

In [102]:
movies.insert_many(data)

In [103]:
movies.find()